In [1]:
# python libraries
import numpy as np
from IPython.display import HTML, display
# pydrake imports
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         Linearize, LinearQuadraticRegulator, LogVectorOutput,
                         MeshcatVisualizer, ModelVisualizer, Parser, Simulator, StartMeshcat)

from underactuated import running_as_notebook

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [105]:
wheel1 = """
<!-- Define the front wheel link and joint -->
  <link name="front_wheel_link">
    <inertial>
      <origin xyz="0 0 1" />
      <mass value="1" />
    </inertial>
    
    <visual>
      <origin rpy="0 1.570796 0" xyz="0 0 0"/>
      <material name="blue">
        <color rgba="0.1 0.1 0.8 1"/>
      </material>
      <geometry>
        <cylinder length="0.05" radius="0.3"/>
      </geometry>
    </visual>
  </link>
  <joint name="front_wheel_joint" type="continuous">
    <parent link="base_link"/>
    <child link="front_wheel_link"/>
    <origin xyz="0 1 0.5"/>
    <axis xyz="-1 0 0"/>
  </joint>
"""

In [106]:
wheel2 = """
  <!-- Define the back wheel link and joint -->
  <link name="back_wheel_link">
    <inertial>
      <origin xyz="0 0 1" />
      <mass value="1" />
    </inertial>
    
    <visual>
      <origin rpy="0 1.570796 0" xyz="0 0 0"/>
      <material name="blue">
        <color rgba="0.1 0.1 0.8 1"/>
      </material>
      <geometry>
        <cylinder length="0.05" radius="0.3"/>
      </geometry>
    </visual>
  </link>
  <joint name="back_wheel_joint" type="continuous">
    <parent link="base_link"/>
    <child link="back_wheel_link"/>
    <origin xyz="0 0 0.5"/>
    <axis xyz="-1 0 0"/>
  </joint>
"""

In [107]:
frame = """
<!-- Define the upper frame link and joint -->
  <link name="top_frame_link">
    <inertial>
      <origin xyz="0 0 1" />
      <mass value="4" />
    </inertial>
    
    <visual>
      <origin rpy="-1.107 0 0" xyz="0 0 0"/>
      <material name="orange">
        <color rgba="0.8 0.4 0.1 1"/>
      </material>
      <geometry>
        <cylinder length="1.118" radius="0.02"/>
      </geometry>
    </visual>
  </link>
  <joint name="top_frame_joint" type="fixed">
    <parent link="base_link"/>
    <child link="top_frame_link"/>
    <origin xyz="0 0.5 0.75"/>
  </joint>
  
  <!-- Define the front frame link and joint -->
  <link name="front_frame_link">
    <inertial>
      <origin xyz="0 0 1" />
      <mass value="1" />
    </inertial>
    
    <visual>
      <origin rpy="0 0 0" xyz="0 0 0"/>
      <material name="orange">
        <color rgba="0.8 0.4 0.1 1"/>
      </material>
      <geometry>
        <cylinder length="0.6" radius="0.02"/>
      </geometry>
    </visual>
  </link>
  <joint name="front_frame_joint" type="fixed">
    <parent link="base_link"/>
    <child link="front_frame_link"/>
    <origin xyz="0 1 0.8"/>
  </joint>
  
  <!-- Define the seat frame link and joint -->
  <link name="seat_frame_link">
    <visual>
      <origin rpy="0 0 0" xyz="0 0 0"/>
      <material name="orange">
        <color rgba="0.8 0.4 0.1 1"/>
      </material>
      <geometry>
        <cylinder length="0.25" radius="0.02"/>
      </geometry>
    </visual>
  </link>
  <joint name="seat_frame_joint" type="fixed">
    <parent link="base_link"/>
    <child link="seat_frame_link"/>
    <origin xyz="0 0.4 0.825"/>
  </joint>
"""

In [108]:
seat = """
  <!-- Define the seat frame link and joint -->
  <link name="seat_link">
    <visual>
      <material name="black">
        <color rgba="0 0 0 1"/>
      </material>
      <geometry>
        <box size="0.1 0.2 0.05"/>
      </geometry>
    </visual>
  </link>

  <joint name="seat_joint" type="fixed">
    <parent link="seat_frame_link"/>
    <child link="seat_link"/>
    <origin xyz="0 0 0.15"/>
  </joint>
"""

In [109]:
handlebar = """
  <!-- Define the handlebar link and joint -->
  <link name="handlebar_link">
    <visual>
      <origin rpy="0 1.570796 0" xyz="0 0 0"/>
      <material name="orange">
        <color rgba="0.8 0.4 0.1 1"/>
      </material>
      <geometry>
        <cylinder length="0.7" radius="0.02"/>
      </geometry>
    </visual>
  </link>
  <joint name="handlebar_joint" type="continuous">
    <parent link="base_link"/>
    <child link="handlebar_link"/>
    <origin xyz="0 1 1.1"/>
    <axis xyz="0 0 1"/>
  </joint>
"""

In [112]:
bicycle_urdf = f"""
<?xml version="1.0" ?>
<robot name="bicycle">

  <!-- Define the base link and joint -->
  <link name="base_link"/>
  <joint name="base_joint" type="fixed">
    <parent link="world"/>
    <child link="base_link"/>
  </joint>
  
  {wheel1}
  {wheel2}
  {frame}
  {seat}
  {handlebar}
</robot>
"""

In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(bicycle_urdf, 'urdf')
visualizer.Run(loop_once=not running_as_notebook)
meshcat.Delete()
meshcat.DeleteAddedControls()

Click 'Stop Running' or press Esc to quit


In [6]:
class TwoWheeledRobotPlant(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("u", BasicVector(2))
        self.DeclareVectorOutputPort("x", BasicVector(3), self.CopyStateOut)
        
        # Define constants
        self.wheel_radius = 0.05
        self.wheel_base = 0.2
        
        # Create the multibody plant
        self.plant = MultiBodyPlant(0.0)
        self.scene_graph = SceneGraph()
        self.plant.RegisterAsSourceForSceneGraph(self.scene_graph)
        self.world_body = self.plant.world_body()
        
        # Add the two wheels as bodies
        self.left_wheel = self.plant.AddRigidBody("left_wheel", self.world_body,
                                                   RigidTransform(RotationMatrix.MakeZRotation(np.pi/2),
                                                                  [self.wheel_base/2, 0, self.wheel_radius]))
        self.right_wheel = self.plant.AddRigidBody("right_wheel", self.world_body,
                                                   RigidTransform(RotationMatrix.MakeZRotation(np.pi/2),
                                                                  [-self.wheel_base/2, 0, self.wheel_radius]))
        
        # Connect the wheels with a fixed joint
        self.plant.AddJoint(
            "fixed",
            self.plant.world_frame(),
            self.left_wheel.body_frame(),
            RigidTransform(),
            self.left_wheel.default_joint_type(),
        )
        self.plant.AddJoint(
            "fixed",
            self.plant.world_frame(),
            self.right_wheel.body_frame(),
            RigidTransform(),
            self.right_wheel.default_joint_type(),
        )

        # Add visual geometry to the wheels
        self.wheel_visual = self.scene_graph.RegisterVisualGeometry(
            self.left_wheel, RigidTransform(),
            self.scene_graph.world_frame(),
            self.scene_graph.CreateCylinder(self.wheel_radius, 2 * self.wheel_radius),
            "wheel_visual", 
            VisualElement(
                cast_shadows=False,
                diffuse_color=np.array([0.5, 0.5, 0.5, 1.0]),
                transparency=0.0,
            )
        )
        
    def CopyStateOut(self, context, output):
        # Get the position and velocity of the two wheels
        q = self.plant.GetPositions(self.plant.GetMyContextFromRoot(context))
        v = self.plant.GetVelocities(self.plant.GetMyContextFromRoot(context))
        
        # Compute the state of the robot
        x = np.array([np.mean([q[self.plant.GetJointByName('left_wheel').position_start()], 
                               q[self.plant.GetJointByName('right_wheel').position_start()]]),
                      0.5 * self.wheel_base * np.sin(q[self.plant.GetJointByName('left_wheel').position_start()] -
                                                    q[self.plant.GetJointByName('right_wheel').position_start()]),
                      np.mean([v[self.plant.GetJointByName('left_wheel').velocity_start()], 
                               v[self.plant.GetJointByName('right_wheel').velocity_start()]])])
        
        output.SetFromVector(x)
        
    def DoCalcTimeDerivatives(self, context, derivatives):
        u = self.EvalVectorInput(context, 0).get_value()
        q = self.plant.GetPositions(self.plant.GetMyContextFromRoot(context))
        
        # Compute the left and right wheel velocities
        left_wheel_vel = u[0] / self.wheel_radius
        right_wheel_vel = u[1] / self.wheel_radius
        
        # Compute the velocities of the two wheels
        v_left_wheel = self.wheel_radius * left_wheel_vel
        v_right_wheel = self.wheel_radius * right_wheel_vel
        
        # Compute the pose of the two wheels
        left_wheel_pose = RigidTransform(RotationMatrix.MakeZRotation(q[self.plant.GetJointByName('left_wheel').position_start()]),
                                         [self.wheel_base/2 * np.cos(q[self.plant.GetJointByName('left_wheel').position_start()]),
                                          self.wheel_base/2 * np.sin(q[self.plant.GetJointByName('left_wheel').position_start()]),
                                          0])
        right_wheel_pose = RigidTransform(RotationMatrix.MakeZRotation(q[self.plant.GetJointByName('right_wheel').position_start()]),
                                          [-self.wheel_base/2 * np.cos(q[self.plant.GetJointByName('right_wheel').position_start()]),
                                           -self.wheel_base/2 * np.sin(q[self.plant.GetJointByName('right_wheel').position_start()]),
                                           0])
        
        # Compute the linear and angular velocities of the robot
        linear_velocity = (v_left_wheel + v_right_wheel) / 2.0
        angular_velocity = (v_right_wheel - v_left_wheel) / self.wheel_base
        
        # Compute the time derivatives of the state
        xdot = np.array([linear_velocity * np.cos(q[self.plant.GetJointByName('left_wheel').position_start()] +
                                                  q[self.plant.GetJointByName('right_wheel').position_start()]) ,
                         linear_velocity * np.sin(q[self.plant.GetJointByName('left_wheel').position_start()] +
                                                  q[self.plant.GetJointByName('right_wheel').position_start()]),
                         angular_velocity])
        
        derivatives.get_mutable_vector().SetFromVector(xdot)

